In [1]:
import json
import pandas as pd

In [ ]:
network = "[Conv2DWorkload(batch=1, height=112, width=112, in_filter=32, out_filter=32, hkernel=3, wkernel=3, hpad=1, wpad=1, hstride=2, wstride=2), Conv2DWorkload(batch=1, height=56, width=56, in_filter=32, out_filter=32, hkernel=1, wkernel=1, hpad=0, wpad=0, hstride=1, wstride=1), Conv2DWorkload(batch=1, height=56, width=56, in_filter=32, out_filter=64, hkernel=1, wkernel=1, hpad=0, wpad=0, hstride=2, wstride=2), ReluConfig(num_filters=64), Conv2DWorkload(batch=1, height=28, width=28, in_filter=64, out_filter=64, hkernel=1, wkernel=1, hpad=0, wpad=0, hstride=1, wstride=1), ReluConfig(num_filters=64), Conv2DWorkload(batch=1, height=28, width=28, in_filter=64, out_filter=160, hkernel=1, wkernel=1, hpad=0, wpad=0, hstride=2, wstride=2), Conv2DWorkload(batch=1, height=14, width=14, in_filter=160, out_filter=400, hkernel=1, wkernel=1, hpad=0, wpad=0, hstride=1, wstride=1), BatchNorm2DConfig(num_filters=400), MaxPool2DConfig(hkernel=3, wkernel=3, hpad=0, wpad=0, hstride=2, wstride=2, ceil_mode=1), Conv2DWorkload(batch=1, height=7, width=7, in_filter=400, out_filter=400, hkernel=3, wkernel=3, hpad=1, wpad=1, hstride=1, wstride=1), ReluConfig(num_filters=400), Conv2DWorkload(batch=1, height=7, width=7, in_filter=400, out_filter=400, hkernel=3, wkernel=3, hpad=1, wpad=1, hstride=1, wstride=1), ReluConfig(num_filters=400), MaxPool2DConfig(hkernel=3, wkernel=3, hpad=0, wpad=0, hstride=2, wstride=2, ceil_mode=1)]"

In [2]:
with open("../profiling_results/ro_convolutions_profiling.json", 'r') as myfile:

    file_data = json.load(myfile)
    
    workloads = file_data['workloads']
            

In [5]:
new_dict = {}
    
for wkl in workloads:
    new_dict[wkl['workload_str']] = wkl

with open("../profiling_results/ro_convolutions_profiling_new_format.json", 'w+') as outfile:
    json.dump(new_dict, outfile, indent=4)

In [9]:
import glob
csv_files = glob.glob(r'ro_conv_profiling/*.csv')

In [7]:
import re

file_re = re.compile(".*\/(conv[\_\d+]+)\_(single|multi)\_sample([\d]+)\.csv")

In [11]:
for file in csv_files:
    match = re.search(file_re, file)
    if match:
        df = pd.read_csv(file)
        data = df['ro_data'].tolist()
        wkl_key = match.group(1)
        exec_string = '{}_exec'.format(match.group(2))
        sample_index = int(match.group(3))
        new_dict[wkl_key]['results'][sample_index][exec_string] = data
        

In [12]:
with open("../profiling_results/ro_convolutions_profiling_new_format.json", 'w+') as outfile:
    json.dump(new_dict, outfile, indent=4)

In [14]:
reduced_new_dict = {}

In [18]:
import numpy as np

In [33]:
overall_max_len_multi = []
overall_max_len_single = []
for wkl in new_dict.keys():
    wkl_dict = new_dict[wkl]
    results = wkl_dict['results']
    multi_lens = []
    single_lens = []
    
    for result in results:
        multi_lens.append(len(result['multi_exec']))
        single_lens.append(len(result['single_exec']))
    max_multi = max(multi_lens)
    max_single = max(single_lens)
    overall_max_len_multi.append(max_multi)
    overall_max_len_single.append(max_single)
    max_multi_index = np.argmax(multi_lens)
    max_single_index = np.argmax(single_lens)
    
    wkl_dict['results'] = {'multi_exec': results[max_multi_index]['multi_exec'],
                           'single_exec': results[max_single_index]['single_exec']
                          }
    reduced_new_dict[wkl] = wkl_dict

In [38]:
with open("../profiling_results/ro_convolutions_profiling_reduced.json", 'w+') as outfile:
    json.dump(reduced_new_dict, outfile, indent=4)